<a href="https://colab.research.google.com/github/binlee52/Individaul-Research-1/blob/Data-Preprocessing/1_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tagging 방법

**태깅** 요소 

상황설명 : 1<br>
등장인물 : 2<br>
대사 : 3<br>
지시사항/감정 : 4<br>
장소 : 5<br>
시간 : 6<br>
시대 : 7<br>
그 외 : 0<br>

In [0]:
# Google drive 연결
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
import re

In [0]:
link = "/content/drive/My Drive/Colab Notebooks/datasets/movie_Action.csv"                  # movie_Action.csv load
index = 192 # 태깅할 영화 선택
data = pd.read_csv(link)                # movie_Action data의 Data Frame 생성
imdbID = data.loc[index, 'imdb_id']     # imdb id 추출

tagging_link = "/content/drive/My Drive/Colab Notebooks/tagged_script/" + imdbID + ".csv"   # tagged script 저장 위치
txt_link = "/content/drive/My Drive/Colab Notebooks/datasets/script_text/" + imdbID + ".txt"
imdbID

'tt0822847'

In [0]:
index = 193
data.loc[index, 'movie_name']

'Program, The'

In [0]:
df = pd.DataFrame(columns=["sentence", "label"])    # tagged scrpt의 Data Frame 생성
df.columns

Index(['sentence', 'label'], dtype='object')

사용할 script를 txt 파일로 변환 및 로드

In [0]:
temp_prepro =(data.loc[index, 'script'])
temp_prepro

In [0]:
#txt 파일로 저장
with open(txt_link, 'w') as f:
   for x in temp_prepro:
       f.write(x)

In [0]:
# txt 파일 불러와서 list로 저장
script = []
with open(txt_link, 'r') as f:
    lines = f.readlines()
    for line in lines:
        script.append(line)

대본 전처리

In [0]:
# script 조작
script = []
script = (data.loc[index, 'script'])
script = script.split("\n")
script = list(filter(lambda x: x!= '', script)) # 공백 제거
# script = script[:10]
script[:10]
temp_re = re.compile('\.\.\.')
temp_re2 = re.compile('^[ ]+$')
for (i, x) in enumerate(script):
    script[i] = re.sub('\.\.\.', ';;;', x)
script_list = []
for x in range(len(script)):
    if not temp_re2.match(script[x]):
        script_list.append(script[x])
script = script_list
#script = script[5:]
script 

태깅

In [0]:
# 정규식
small_re = re.compile('[a-z]+')                     # 소문자가 있는지 판별
situation_re = re.compile(' {15}[\S]+')                  # 상황설명 1 
person_re = re.compile(" {37}[^a-z ]+[^a-z]*$")      # 등장인물 2
line_re = re.compile(' {25}[\S]+')                   # 대사 3
command_re = re.compile(' {30}[\S]+')              # 감정/지시 4 # ' '*
scence_re = re.compile(' {15}[A-Z]+[^a-z]*$')        # 씬 5, 6, 7
in_re = re.compile('[^a-z]*INT.')
out_re = re.compile('[^a-z]*EXT.')                  # 장소(내부/외부) 5
#time_re = re.compile(' - [A-Z]+$')                  # 시간 6
time_re = re.compile(' - [A-Z]+[\-]*[A-Z]*$')
fuck_re = re.compile('[--]$')
t_re = re.compile('\-')                              # 시대 7
t2_re = re.compile('^ {10}[A-Z]+[A-Z ]*$')
#                                                   # 그외 0
#
#------------------------------------------------------------------------------
sentence_cnt = 0    # Data Frame의 index로 사용
is_time = False     # 시간에 관한 데이터가 장소에 대한 data 보다 나중에 입력되지 않게 하기위해 사용

In [0]:
# 태깅 진행

for sc in script:
    if small_re.search(sc):                                 # 소문자 판별
        if line_re.match(sc):                               # 대사 3           
            df.loc[sentence_cnt, 'sentence'] = sc.strip()
            df.loc[sentence_cnt, 'label'] = 3
        elif command_re.match(sc):                          # 지시/감정 4
            df.loc[sentence_cnt, 'sentence'] = sc.strip()
            df.loc[sentence_cnt, 'label'] = 4
        elif situation_re.match(sc):                        # 상황설명 1
            df.loc[sentence_cnt, 'sentence'] = sc.strip()
            df.loc[sentence_cnt, 'label'] = 1
        else:                                               # 기타 0
            df.loc[sentence_cnt, 'sentence'] = sc
            df.loc[sentence_cnt, 'label'] = 0
    elif t2_re.match(sc) and not t_re.search(sc):        
        df.loc[sentence_cnt, 'sentence'] = sc.strip()
        df.loc[sentence_cnt, 'label'] = 0
    else:
        if fuck_re.search(sc):
            df.loc[sentence_cnt, 'sentence'] = sc.strip()
            df.loc[sentence_cnt, 'label'] = 0
        elif line_re.match(sc):                               # 대사 3
            df.loc[sentence_cnt, 'sentence'] = sc.strip()
            df.loc[sentence_cnt, 'label'] = 3
        elif person_re.match(sc):                           # 등장인물 2             
            df.loc[sentence_cnt, 'sentence'] = sc.strip()
            df.loc[sentence_cnt, 'label'] = 2    
        elif scence_re.match(sc):                           # 씬 5, 6, 7
            if in_re.match(sc):                             # INT. 5           
                side = in_re.match(sc).group(0)
                df.loc[sentence_cnt, 'sentence'] = side
                df.loc[sentence_cnt, 'label'] = 5
                sentence_cnt += 1
                sc = sc.replace(side, "").strip()
            elif out_re.match(sc):                          # EXT. 5
                side = out_re.match(sc).group(0)
                df.loc[sentence_cnt, 'sentence'] = side
                df.loc[sentence_cnt, 'label'] = 5
                sentence_cnt += 1
                sc = sc.replace(side, "").strip()
            if time_re.search(sc):                          # 시간 6
                sentence_cnt += 1
                time = time_re.search(sc).group(0)
                sc = sc.replace(time, "").strip()
                time = time.replace("-", " ").strip()        # '-' 삭제
                df.loc[sentence_cnt, 'sentence'] = time
                df.loc[sentence_cnt, 'label'] = 6
                sentence_cnt -= 1
                is_time = True    
            df.loc[sentence_cnt, 'sentence'] = sc
            df.loc[sentence_cnt, 'label'] = 5
            if is_time:
                sentence_cnt+=1
                is_time = False
        else:                                               # 기타 0
            df.loc[sentence_cnt, 'sentence'] = sc
            df.loc[sentence_cnt, 'label'] = 0
    sentence_cnt += 1

In [0]:
df.loc[0, 'label'] = 0
#df.loc[5249, 'label'] = 0
df = df.sort_index() # 정렬
df.loc[:]

,sentence,label
0,FADE IN,0
1,ON BLACK SCREEN,5
2,SPORTSCASTER'S VOICE (V.O.),2
3,"Allright, bowl bid ridin' here for",3
4,E.S.U. Fourth down on the Georgia,3
...,...,...
5021,on to;;;,3
5022,They continue down the tunnel toward a new rec...,1
5023,another year in the annals of The Program.,1
5024,...,0


In [0]:
# 결과
df = df.sort_index() # 정렬

for x, row in enumerate(df.iterrows()):
    if df.loc[x, 'label'] == 0:
        print(df.loc[x, 'sentence'], "                  index of data >> ", x)

               FADE IN                   index of data >>  0
                                                            INSTANT CUT TO:                   index of data >>  7
               $400,000,000.                   index of data >>  108
                                                               DISSOLVE TO:                   index of data >>  608
                                                              SMASH CUT TO:                   index of data >>  849
                                                              SMASH CUT TO:                   index of data >>  1140
                                                                  FADE OUT:                   index of data >>  1198
                                                                    CUT TO:                   index of data >>  1470
                                                              SMASH CUT TO:                   index of data >>  1868
                                                        

In [0]:
# 예외처리
df.loc[108, 'label'] = 1

In [0]:
# 인물 명 내에 괄호 제거 -> 고치기
p_re = re.compile('\([^\)]+\)')
for x, row in enumerate(df.iterrows()):
    if df.loc[x, 'label'] == 2:
        if p_re.search(df.loc[x, 'sentence']):
            df.loc[x, 'sentence'] = p_re.sub("", df.loc[x, 'sentence']).strip()

In [0]:
df.loc[:30]

,sentence,label
0,FADE IN,0
1,ON BLACK SCREEN,5
2,SPORTSCASTER'S VOICE,2
3,"Allright, bowl bid ridin' here for",3
4,E.S.U. Fourth down on the Georgia,3
5,Tech 18. 4 ticks left. Kane sets 'em,3
6,down.,3
7,...,0
8,LINE OF TAPED KNUCKLES,5
9,NIGHT,6


데이터 저장

In [0]:
df.to_csv(tagging_link, index = False)